# Train HTR recognition models with Kraken

This notebook assumes the following folder structure: `data` for ground truth, `mod` for saving the models gained through training, and `src` containing this notebook and potential base models.

There are basically two ways of training HTR modells with Kraken: You can train a model *from scratch* or *fine-tune* a pre-trained model. Fine-tuning a modell to your specific needs is more ressource-efficient. It will go faster and you can achieve good results with a relatively small amount of ground truth. Training a modell from scratch requires more effort but your modell will be well fit to your data. This notebook will take you through both of these options, make sure you only follow the sections needed for your project.



## Installation
For either option first install Kraken `kraken`. This may take a few minutes.

If you wish to use Kraken's data augmentation feature also install `albumenations`.

In [ ]:
!pip install kraken > pip_output.txt
!pip install albumentations

## Prepare training directory
Before you start training you need to upload the data you would like to train your model on. This includes 
- image files,
- corresponding transcriptions as xml-files (PAGE or ALTO),
- for fine-tuning also: a pre-trained model.

If you are not using the suggested directory structure adjust the `PATH` in the line below. Otherwise run it directly.

In [ ]:
!find ../data -type f -name "*.xml" > output.txt

Now adjust the split for your ground truth. `0.8 0.1 0.1` corresponds to the training data being split into a training set (80%), a validation set (10%) to check results at the end of each epoch and a test set (10%) for final validation of the new model. Adjust the split if necessary.

If your ground truth comes in PAGE xml, change `alto` to `page`.

In [ ]:
!ketos compile -F output.txt --random-split 0.8 0.1 0.1 -f alto

## Training a model from scratch

Use the following line to train a model **from scratch**. If you wish to fine-tune a model, skip to the next section.
The following line provides a path to where your models are going to be stored. By default it has been set as `output`. Adjust `PREFIX_NEW_MODEL` to a prefix of your choice.

The following line utilises GPU for training. If you wish to run training on CPU delete `-d cuda:0`. It also provides data augmentation `--augment` and a minimum of 50 training epochs. If needed, change the parameter. Then run this line.

In [ ]:
!ketos train -f binary -d cuda:0 -o ../output/PREFIX_NEW_MODEL --augment --min-epochs 50 dataset.arrow

Training will take quite some time while Kraken keeps saving model files to the assigned folder. 

When training ends, choose the model best suited and download it. Also, download whatever files you may decide to keep as *Colab* will delete all of your data by default.

## Fine-tune a pre-trained model

If you wish to use a base model and fine-tune that with your own data, save that model to `data`. Now adjust the path to your base model `MODEL.mlmodel` and add the prefix (i.e., the name of yourmodel without `.mlmodel`) for your fine-tuned model. Then run this line.

In [ ]:
!ketos train -f binary -d cuda:0 --resize both -i ../data/MODEL.mlmodel -o ../mod/PREFIX_NEW_MODEL --augment --min-epochs 50 dataset.arrow

Training will take quite some time while Kraken keeps saving model files to the assigned folder.

When training ends, choose the model best suited and download it. Also, download whatever files you may decide to keep as *Colab* will delete all of your data by default.